# Portfolio Project: Online Retail EDA with Python

# Project Overview  

This project explores an **online retail transactions dataset**, focusing on **data cleaning, exploratory data analysis (EDA), and deriving business insights**. The dataset contains information about customer purchases, including invoice details, product descriptions, quantities, prices, and customer IDs. 


### Objectives  
- Perform **Exploratory Data Analysis (EDA)** to identify key trends.  
- Analyze **sales performance, customer behavior, and popular products**.  
- Provide **data-driven recommendations** to optimize online retail strategies.  


### My Approach  
To tackle this project, I’ll start by **ETL (Extract, Transform, Load)** to clean and prepare the dataset. Then, I’ll conduct in-depth analysis to identify key trends and insights like **busiest sales periods, top-selling products, and high-value customers**. Let's dive in!  


# Dataset Overview


For this project, I'll be working with the **Online Retail** dataset, which contains transactional data from an online store between 2010 and 2011. The dataset is in a `.csv` file named **`online_retail.csv`**, and it includes details about purchases such as product descriptions, quantities, prices, timestamps, and customer IDs.  


### Data Columns  
The dataset consists of the following fields:  
- **InvoiceNo** – Unique invoice number for each transaction.  
- **StockCode** – Unique product identifier.  
- **Description** – Product name/description.  
- **Quantity** – Number of units purchased.  
- **InvoiceDate** – Timestamp of the transaction.  
- **UnitPrice** – Price per unit of the product.  
- **CustomerID** – Unique identifier for each customer.  
- **Country** – Country where the transaction took place.  


### My Approach  

To analyze this dataset effectively, I’ll break the process into key steps:  

1. **Load the data** into a Pandas DataFrame and inspect the first few rows.  
2. **Clean the dataset** by handling missing values and removing unnecessary columns.  
3. **Explore basic statistics** to understand distributions and trends.  
4. **Visualize the data** using plots such as histograms, bar charts, and scatter plots.  
5. **Analyze sales trends** over time to identify peak sales periods.  
6. **Identify top-selling products and countries** based on quantity sold.  
7. **Detect anomalies or outliers** that may impact the analysis.  
8. **Summarize key findings** and insights from the data.  

Let's dive in and explore the dataset!  


# ETL

## 01. Load the data

Import the required libraries and load the dataset.

In [1]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

df = pd.read_csv("source/online_retail.csv", encoding="ISO-8859-1")  # We use encoding to avoid UnicodeDecodeError (or encoding="Windows-1252")

Explore and familiarize with the dataset

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [3]:
df.describe()

# TODO: Found some negative values in quantity and unit price. We need to check if they are valid or not.

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


## 02. Clean the dataset

Now that we have identified the data types of each column and detected any missing (null) values, we have a clearer understanding of how to approach the ETL process.

Before proceeding, let's create a copy of the dataframe to preserve the original data in its unaltered state.

In [4]:
df_clean = df.copy()

### Type Casting

With the copy created, we will begin by modifying the data types of specific columns.  
In this case, we will convert the `Country`, `InvoiceNo`, and `StockCode` columns from the object type to the category type.  
This transformation will optimize memory usage and improve performance when handling these columns in Pandas.


In [5]:
df_clean['Country'] = df['Country'].astype('category')
df_clean['InvoiceNo'] = df['InvoiceNo'].astype('category')
df_clean['StockCode'] = df['StockCode'].astype('category')

# Ensure the data types where set correctly with: df_clean.info()

### Handling Missing Values

#### Handling missing `CustomerID` values

The dataset contains missing values in the `CustomerID` column, but these transactions are still valid purchases. Instead of dropping them or imputing arbitrary values (which could introduce bias), I will leave them as `NaN`.

 Why?
- Removing these rows would result in **loss of actual transaction data**.
- Imputing fake IDs would be **misleading**, as customer IDs are unique identifiers.
- Pandas and Matplotlib **handle NaN values gracefully** in most operations.

#### Handling missing `Description` values

The dataset contains null values in the `Description` column. Since these rows cannot be dropped without losing valuable data, we impute the missing descriptions using the corresponding `StockCode` values (which are complete and unique).

For that purpose, we follow this steps:
1. **Create a mapping dictionary** where each `StockCode` points to its correct `Description` (using only rows with non-null descriptions)
2. **Fill null values** by matching each missing `Description` with its `StockCode`'s known description

**Key Note**: If a `StockCode` has no valid description in the dataset, its `NaN` values will remain.

In [6]:
# Step 1: Map StockCode to Description (drop duplicates to ensure 1:1 mapping)
stock_to_desc = df_clean.dropna(subset=['Description']).drop_duplicates('StockCode').set_index('StockCode')['Description']

# Step 2: Fill NaN Descriptions using the mapped StockCode values
df_clean['Description'] = df_clean['Description'].fillna(df_clean['StockCode'].map(stock_to_desc))

After handling the preliminary missing values in the `Description` column, it's important to verify if any null values still remain. We will perform this check to ensure that all missing descriptions have been properly handled before moving forward with further analysis.

To do so, we'll check for any remaining nulls in the column.

In [7]:
# This will give us an updated count of the missing values in the 'Description' column
print(f'Original Description column null values: {df['Description'].isna().sum()}')
print(f'Updated Description column null values: {df_clean['Description'].isna().sum()}')

Original Description column null values: 1454
Updated Description column null values: 112


##### Imputing Remaining Null Values

After checking for null values, we found that 112 missing descriptions remain out of the initial 1,454 null values. To ensure we don't lose valuable transaction data, we will impute these remaining null values with the placeholder `'Unknown'`. This decision allows us to retain all rows in the dataset while clearly marking the transactions with missing descriptions.

In [8]:
df_clean['Description'] = df_clean['Description'].fillna('Unknown')

# To make sure this worked as intended: print(df_clean['Description'].isnull().sum())

By doing this, we preserve the full dataset while handling missing descriptions in a way that keeps the integrity of our analysis intact.

### String Processing

From here forward, I need to write better the markdown documentation.

As we intend to remove duplicates in the next step, we first need to make sure that Pandas recognizes all duplicated rows as so. For that, we'll make sure to that all the strings are consistent and trimmed. 

For that purpose, we'll manage mainly 2 columns: `Description` and `Country`, since they are the only ones with string type data. 
For `Description`, we'll trim the leading, trailing, and in-between extra whitespaces, and standarize it to all uppercase
For `Country`, we'll also trim the leading, trailing, and in-between extra whitespaces, and standarize them all as titles, with the first letter as uppercase, and the rest as lowercase

In [9]:
# Clean Description: Remove leading/trailing spaces, handle in-between extra spaces, and standardize to lowercase
df_clean['Description'] = df_clean['Description'].str.strip().str.replace(r'\s+', ' ', regex=True).str.upper()

# Clean Country: Remove leading/trailing spaces, handle in-between extra spaces, and title-case the country names
df_clean['Country'] = df_clean['Country'].str.strip().str.replace(r'\s+', ' ', regex=True).str.title()

### Removing Duplicates

Any duplicated values may generate bias and result in false insights drawn from the dataset, so it's necessary to deal with them. For that, we start by checking if there is any duplicated data to handle.

Since all the columns can have duplicated values individually, we just need to drop the rows that have duplicate values in all the columns at once. So we can easily use Pandas .drop_duplicates() method to do so.

In [10]:
print(f'Number of duplicate rows: {df_clean.duplicated().sum()}')

Number of duplicate rows: 5268


In [11]:
# Remove exact duplicate rows
df_clean = df_clean.drop_duplicates()

### Validating Negative Values

Based on the results of df.describe() in the 'Load the data' step, we found some negative values in the `Quantity` and `UnitPrice` columns. Since they are not supposed to be there, we'll handle them by...

In [12]:
# Check for negative values in Quantity and UnitPrice
print(df_clean[df_clean['Quantity'] < 0].shape[0])
print(df_clean[df_clean['UnitPrice'] < 0].shape[0])

10587
2


Since `Quantity` negative values are substantially higher, we'll start by handling them

#### Analizing the negative values in `Quantity`

In [13]:
# Take a look at the negative values in Quantity and look for patterns
df_clean[df_clean['Quantity'] < 0].sample(20)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
206139,C554905,15058A,BLUE POLKADOT GARDEN PARASOL,-2,5/27/2011 11:17,7.95,14191.0,United Kingdom
466258,C576324,23470,CARD HOLDER LOVE BIRD LARGE,-1,11/14/2011 15:10,6.25,16161.0,United Kingdom
349892,C567543,23346,SPACEBOY BEAKER,-6,9/21/2011 10:19,1.25,13418.0,United Kingdom
221643,556260,37340,SOLD AS SET ON DOTCOM,-18,6/9/2011 18:02,0.00,NaN,United Kingdom
176843,C552029,20982,12 PENCILS TALL TUBE SKULLS,-1,5/5/2011 18:11,0.85,15622.0,United Kingdom
72926,C542273,22099,CARAVAN SQUARE TISSUE BOX,-9,1/27/2011 10:01,1.25,12578.0,Italy
20393,538042,21763,VINTAGE WOODEN BAR STOOL,-4,12/9/2010 13:10,0.00,NaN,United Kingdom
467904,C576375,22583,PACK OF 6 HANDBAG GIFT BOXES,-1,11/15/2011 8:52,2.55,16878.0,United Kingdom
437426,C574291,22633,HAND WARMER UNION JACK,-1,11/3/2011 15:19,2.10,15290.0,United Kingdom
79477,C542991,85019C,CURIOUS IMAGES NOTEBOOK SET,-12,2/2/2011 11:54,4.25,14460.0,United Kingdom


We can notice some patterns:
1. Most of the InvoiceNo values have a C at the beginning, probably meaning they are "Credit"
2. Some Description values are special cases, like "DAMAGED", "DISCOUNT", or "?"

Because of that, let's see how many unique descriptions are associated with negative quantities:

In [14]:
negative_descriptions = df_clean[df_clean['Quantity'] < 0]['Description'].value_counts()
print(negative_descriptions.count())

2471


Since there are so many negative unique values, we can look for the most frequently used ones.

In [15]:
print(negative_descriptions.head(30))  # Show the top 30 most frequent negative descriptions

Description
MANUAL                                 244
REGENCY CAKESTAND 3 TIER               180
POSTAGE                                126
CHECK                                  123
UNKNOWN                                 97
JAM MAKING SET WITH JARS                87
DISCOUNT                                77
SET OF 3 CAKE TINS PANTRY DESIGN        75
SAMPLES                                 61
DAMAGED                                 57
STRAWBERRY CERAMIC TRINKET BOX          54
ROSES REGENCY TEACUP AND SAUCER         54
RECIPE BOX PANTRY YELLOW DESIGN         47
DAMAGES                                 46
JUMBO BAG RED RETROSPOT                 44
LUNCH BAG RED RETROSPOT                 44
WOOD 2 DRAWER CABINET WHITE FINISH      43
RED RETROSPOT CAKE STAND                42
WHITE HANGING HEART T-LIGHT HOLDER      42
GREEN REGENCY TEACUP AND SAUCER         42
?                                       42
SMALL GLASS HEART TRINKET POT           40
SET OF 3 REGENCY CAKE TINS              37

#### Handling the negative values in `Quantity`

The output shows that some `Description` values represent regular products, while others indicate special cases (discounts, damaged products, or ambiguous values like "?"). We implement a unified classification system:

1. Categorize All Transactions

    Create a `TransactionType` column with three distinct labels:

    * **Return:** Transactions where `InvoiceNo` starts with "C" (credit notes).
    * **SpecialCase:** Transactions with descriptions matching predefined non-product terms (DISCOUNT, DAMAGED, SAMPLES, ?, etc.).
    * **Sale:** All other regular transactions.

2. Process Negative Quantities

    * **Returns:** Keep negatives (valid refund records).
    * **Special Cases:** Preserve original values (context-dependent).
    * **Sales:** Convert negatives to positives (assumed data entry errors).

**Note:** To distinguish legitimate negative quantities from data entry errors:

* Identified the top 30 most frequent descriptions for negative quantities

* Manually selected non-product terms (e.g., `DISCOUNT`, `DAMAGED`, `?`)

Resulting in the curated special_case_list used for classification.

In [16]:
# Predefined list of special cases for descriptions
special_case_list = [
    'DISCOUNT', 'DAMAGED', 'DAMAGES', 'SAMPLES', 'CHECK', 'MANUAL', 'POSTAGE', 
    'UNKNOWN', '?', 'AMAZON FEE'
]

In [17]:
def classify_transaction(invoice_no, description):
    """Classify transaction as 'Return', 'SpecialCase', or 'Sale'."""
    if str(invoice_no).startswith('C'):
        return 'Return'
    elif description in special_case_list:
        return 'SpecialCase'
    else:
        return 'Sale'

# Use .map() efficiently by applying it on a tuple of (InvoiceNo, Description)
df_clean['TransactionType'] = list(map(classify_transaction, df_clean['InvoiceNo'], df_clean['Description']))

In [18]:
# Convert negatives to positives ONLY for regular Sales
df_clean.loc[
    (df_clean['TransactionType'] == 'Sale') & 
    (df_clean['Quantity'] < 0), 
    'Quantity'
] = df_clean['Quantity'].abs()  # Or: *= -1

To verify this approach worked as intended, let's see if there are any negative values left that are not returns or special cases:

In [25]:
print(df_clean[
    (df_clean['Quantity'] < 0) & 
    (~df_clean['TransactionType'].isin(['Return', 'SpecialCase']))
].shape[0])

0


Since the result is 0, it means we correctly handled the negative values and can continue to the next step.

#### Analizing and handling the negative values in `UnitPrice`

As seen before, there are just 2 transactions with negative UnitPrice. Therefore, we can search for them directly as follows:

In [27]:
df_clean[df_clean['UnitPrice'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TransactionType
299983,A563186,B,ADJUST BAD DEBT,1,8/12/2011 14:51,-11062.06,NaN,United Kingdom,Sale
299984,A563187,B,ADJUST BAD DEBT,1,8/12/2011 14:52,-11062.06,NaN,United Kingdom,Sale


Since they are labeled as "ADJUST BAD DEBT", these appear to represent financial adjustments rather than product sales.  

These transactions are valid records, so we will keep them in the dataset and classify them under "SpecialCase" in the `TransactionType` column while preserving their negative values.

In [28]:
# Ensure "ADJUST BAD DEBT" transactions are marked as SpecialCase
df_clean.loc[df_clean['Description'] == 'ADJUST BAD DEBT', 'TransactionType'] = 'SpecialCase'

### Handling Outliers